# Project 101
### PARTE 1 - EDA

### Contenido
- [Introducción](#intro)
- [Extracción de datos](#1)
- [EDA](#eda)

    - [Análisis de kilos](#2)
    - [Comparación Impo vs Expo](#3)
    - [Porcentaje de variación mensual](#4)
    - [Comparativo Kilos Semanales por Servicio](#5)
    - [Variables Exógenas que afectan el análisis](#6)


### Introducción <a id='intro'></a>

En esta primera sección del proyecto el objetivo es lograr obtener, limpiar y realizar un analisis descriptivo de los datos extraídos de la base de datos. De esta forma, buscamos comprender y analizar de forma exhaustiva el comportamiento de las variables de interés a lo largo del tiempo, para luego en la siguiente sección, hallar el modelo que mejor interprete la relacinó de las variables en el tiempo y logre predecir su comportamiento futuro. 

Para este proyecto, utilizaremos la base de datos de Raiconet, la cual se encuentra dentro de una VPN. Dentro de esta base de datos, se ecuentran todas las tablas del sistema de gestion, courier y contable llamado "Raquel". De esta base de datos Microsoft SQL, nuestro analisis estará basado en la table GUIAS la cual consta de entradas de todas las guias procesadas desde el 2019 hasta la fecha. A su vez, como se observara en el query a continuación utilizaremos otras tablas anexas para obtener detalles sobre los clientes, servicios, rutas y demas detalles que serán utiles en el análisis. Así, es posible analizar toda la prestación de servicios de los últimos 5 años y analizar ciclos, ventas, servicios, vendedores, mejores clientes, por kilos y por momentos del año. 

EL objetivo de este proyecto es lograr realizar un modelo de serie de tiempo que prediga la cantidad de kilos que se van traer en los proximos 3 meses, para cada motivo de facturacion (servicio), por ello esta primer etapa es crucial en comprender mejor la informmación con la que trabajaremos. 

In [68]:
import pandas as pd
from database_credentials import UID, PWD
from geopy.geocoders import Nominatim
import pyodbc 
import numpy as np
from plotly.subplots import make_subplots
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
from matplotlib import pyplot as plt
import plotly.io as pio
pio.renderers
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

### Extracción de datos <a id='1'></a>

##### Conexion a DB

In [69]:
cnxn_str= ("Driver={SQL Server};"
            "Server=172.16.2.49;"
            "Database=Raico;"
            f"UID={UID};"
            f"PWD={PWD};")
cnxn = pyodbc.connect(cnxn_str)

In [70]:
query_guias= '''
    SELECT [Guias_Id]
      ,[Guias_Numero]
      ,[Guias_Fecha]
      ,Guias.Motivos_Id
      ,[Proveedores_Id]
      ,[Guias_ContraGuia]
      ,[Guias_GuiaMadre]
      ,[Guias_Tracking]
      ,[Paises_Id]
      ,[Paises2_Id]
      ,[Ciudades_Id]
      ,[Ciudades2_Id]
      ,[ZonasV_Id]
      ,[ZonasIV_Id]
      ,[Contenidos_Id]
      ,[Guias_Bultos]
      ,[Guias_Peso]
      ,[Guias_VolLargo]
      ,[Guias_VolAncho]
      ,[Guias_VolAlto]
      ,[Guias_PesoVol]
      ,[Guias_FOB]
      ,Guias.Clientes_Id
      ,[Guias_ExpoTotVta]
      ,[Guias_ImpoTotVta]
      ,[Guias_VentaI]
  FROM [Raico].[dbo].[Guias]
  WHERE Guias_Fecha> '2019' 
  '''
df_guias = pd.read_sql_query(query_guias, cnxn)

query_motivos= '''
select Motivos_Id, MotivosFac.MotivosFac_Id, MotivosFac_Nombre
from motivos
inner join MotivosFac
on MotivosFac.MotivosFac_Id= Motivos.MotivosFac_Id
'''
df_motivos= pd.read_sql_query(query_motivos, cnxn)


query_paises= '''
select Paises_Id, Paises_Codigo, Paises_Nombre
from Paises
'''
df_paises= pd.read_sql_query(query_paises, cnxn)

quey_ciudades= '''
select Ciudades_Id, Ciudades_Nombre
from Ciudades
'''
df_ciudades= pd.read_sql_query(quey_ciudades, cnxn)

query_proveedores= '''
select Proveedores_Id, Proveedores_Nombre
from Proveedores
'''
df_proveedores= pd.read_sql_query(query_proveedores, cnxn)

query_clientes= '''select Clientes_Id, Clientes_Nombre
from clientes'''
df_clientes= pd.read_sql_query(query_clientes, cnxn)

c:\Users\Bravo15\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



c:\Users\Bravo15\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

c:\Users\Bravo15\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

c:\Users\Bravo15\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

c:\Users\Bravo15\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 ob

#### EDA <a id='eda'></a>

In [71]:
df_guias.head()

,Guias_Id,Guias_Numero,Guias_Fecha,Motivos_Id,Proveedores_Id,Guias_ContraGuia,Guias_GuiaMadre,Guias_Tracking,Paises_Id,Paises2_Id,...,Guias_Peso,Guias_VolLargo,Guias_VolAncho,Guias_VolAlto,Guias_PesoVol,Guias_FOB,Clientes_Id,Guias_ExpoTotVta,Guias_ImpoTotVta,Guias_VentaI
0,404102,996-10129556,2019-08-30,160,5298.0,295-00001135,None,None,NaN,32.0,...,144.0,None,None,None,192.0,1528.20,42417,1298.1,NaN,0.00
1,406822,US00192167,2019-01-02,36,1830.0,None,None,None,91.0,NaN,...,13.0,None,None,None,2.0,237.50,33511,NaN,256.70,161.36
2,406824,I180974591,2019-01-02,128,4165.0,None,None,None,25.0,NaN,...,23.0,None,None,None,11.0,1785.78,33511,NaN,1150.81,434.01
3,406825,US00192152,2019-01-02,36,1830.0,None,None,None,91.0,NaN,...,1.0,None,None,None,1.0,84.65,30148,NaN,131.69,92.83
4,406826,RM00191696,2019-01-02,51,1870.0,None,725202626,None,76.0,NaN,...,32.0,None,None,None,35.0,1316.22,36568,NaN,1161.37,567.92


Para el análisis de los kilos transportados, los kilos son medidos en "Kilos Facturables" que es el resultado entre el máximo entre el peso del envio y el peso volumétrico. Esto se debe a la naturaleza del negocio y a la forma en la que las compañías aereas cobran por el espacio en bodega y a su vez, la forma en la que los partners logísticos en las distintas sedes cobran por los pickups y transporte local. 

In [72]:
df_guias['kilos_facturables']= df_guias[["Guias_Peso", "Guias_PesoVol"]].max(axis=1)

In [73]:
df_guias[['Guias_Peso', 'Guias_PesoVol', 'kilos_facturables']].head()

,Guias_Peso,Guias_PesoVol,kilos_facturables
0,144.0,192.0,192.0
1,13.0,2.0,13.0
2,23.0,11.0,23.0
3,1.0,1.0,1.0
4,32.0,35.0,35.0


In [74]:
df_guias.shape

(141279, 27)

In [75]:
df_guias.describe()

,Guias_Id,Motivos_Id,Proveedores_Id,Paises_Id,Paises2_Id,Ciudades_Id,Ciudades2_Id,ZonasV_Id,Contenidos_Id,Guias_Bultos,Guias_Peso,Guias_PesoVol,Guias_FOB,Clientes_Id,Guias_ExpoTotVta,Guias_ImpoTotVta,Guias_VentaI,kilos_facturables
count,141279.000000,141279.000000,139271.000000,99293.000000,41948.000000,40071.000000,99618.000000,78968.000000,141279.000000,140823.000000,140788.000000,134492.000000,1.338180e+05,141279.000000,42406.000000,98708.000000,141279.000000,140806.000000
mean,482366.302324,93.926174,2256.161118,76.620134,64.167708,30.368770,2.854263,113.659204,2.926330,1.871704,19.948899,19.317211,1.062406e+03,37723.004657,192.439414,862.483145,255.083124,24.337335
std,43686.696214,52.194997,994.685803,29.858044,39.824973,80.618387,14.509737,106.275417,0.376714,6.142031,51.082843,59.828459,7.683175e+03,4900.078453,333.867240,2494.369691,313.281539,66.336982
min,404102.000000,24.000000,1226.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,0.000000,0.000000,1.000000e-01,27948.000000,3.200000,0.720000,0.000000,0.000000
25%,444111.500000,36.000000,1830.000000,76.000000,24.000000,2.000000,2.000000,76.000000,3.000000,1.000000,2.000000,4.500000,1.198500e+02,33511.000000,56.585000,456.412500,0.000000,4.500000
50%,482971.000000,120.000000,1830.000000,91.000000,91.000000,2.000000,2.000000,99.000000,3.000000,1.000000,9.000000,10.000000,7.097850e+02,39416.000000,105.900000,744.795000,167.800000,12.000000
75%,519975.500000,137.000000,2189.000000,91.000000,91.000000,2.000000,2.000000,101.000000,3.000000,2.000000,28.000000,21.000000,1.240820e+03,41217.000000,197.860000,1148.547500,391.225000,32.000000
max,557914.000000,177.000000,7638.000000,254.000000,311.000000,509.000000,510.000000,729.000000,3.000000,510.000000,4338.000000,8304.000000,1.539440e+06,46040.000000,12682.130000,476854.950000,25155.590000,8304.000000


##### Análisis de kilos <a id='2'></a>

Agrupamos los datos por mes para observar a grandes rasgos como es la distribucion mensual del total de kilos facturados en el periodo.

In [76]:
df_guias_group_mensual= df_guias.groupby(pd.Grouper(key='Guias_Fecha', axis=0, freq='M')).sum()

In [77]:
fig = px.line(df_guias_group_mensual, x=df_guias_group_mensual.index, y='Guias_Peso', title='Total Kilos por Mes', )

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

fig.show()

Con este tipo de gráfica no solo se puede observar la evolución general de los kilos transportados por mes sino tambien el outlier que se encuentra en diciembre 2023. Ademas observamos que mensualmente la evolución de kilos transportados posee variaciones y las tendencias varian año a año, aunque se observa claramente el impacto de la pandemia en el 2020 y como luego de unos meses comenzo a aumentar el volumen de kilos transportados. Por otro lado, se observa que el maximo de kilos transportados fue mas de 68.000 en agosto de 2022, y el segundo maximo fue en mayo de 2023 con mas de 67.800 kilos. 

Si comparamos el comienzo del 2023 con el 2022, observamos como el año anterior fue mucho mas estable la evolución de kilos de carga, mientras que este año presentó mayores fluctuaciones de kilos entre los meses con una variación aproximada de +-30%. Esto se puede deber a la inestabilidad política y económica que presento esto año en particular, con las elecciones presidenciales en Octubre y con una volatilidad del dólar mucho mayor a la del año anterior. Este hecho repercute de forma directa en la capacidad de los clientes para importar mercadería para comercializar o exportar sus productos. Debemos considerar que la suba del dolar no es aislada a mayores restricciones en aduana y complicaciones para pagar en el exterior lo que perjudica la capacidad de compra, y pagos en el exterior que no solo afecta a los clientes, ergo a los kilos transportados, sino tambien pone en riesgo la capacidad de Raico de pagarle a sus proveedores en el exterior, principalmente las compañías aéreas (Aerolineas Argentinas, Iberia, American Airlines, IFS), agentes (UPS, FEDEX, Geobox) o incluso los pickups domésticos en Europa y Asia. 


##### Comparación Impo vs Expo <a id='3'></a>

El negocio se divide en dos sectores principales ya que si bien los servicios logísticos son parecidos, su gestion y procesos son realizados por separado. De esta forma, la empresa se divide en dos grandes sectores: Importación y Exportación. Los procesos de ambos estan ilustrados debajo para mayor comprension del análisis, pero en resumidas cuentas son inversos entre si y en ambos intervienen los mismo agenntes. Aun asi por politicas gubernamentales, y de acuerdo al partido que gobierne es el regimen que se vera favorecido. Si comparamos impo vs expo, se observa que durante la presidencia de Macri el regimen de exportacion se vio favorecido lo que represento mayor protagonismo de las exportaciones sobre las importaciones, pero en cambio con la llegada del nuevo regimen en 2019, el regimen de importación logró mayor popularidad representando hoy mas de 80% de las ventas y logrando records en la cantidad de kilos transportados para este servicio. 

Al ser un negocio fuertemente vinculado con los regimenes aduaneros que varian drasticamente de acuerdo a las politicas economicas o de gobierno, la adaptabilidad es un factor clave, donde la posibilidad de prestar ambos servicios es una gran ventaja ya que cuando uno de ellos es desfavorecido el segundo no lo es, por lo que permite orientar rapidamente la estrategia hacia el sector no afectado, de esta forma es un negocio que se amolda a las necesidades de los clientes y a las condiciones del pais, permitiendo sobrepasar tormentas y momentos donde la logistica es fuertemente afectada. 


In [78]:
df_guias_expo= df_guias.dropna(subset=['Guias_ExpoTotVta'])
df_guias_impo= df_guias.dropna(subset=['Guias_ImpoTotVta'])

In [79]:
df_guias_expo.describe()

,Guias_Id,Motivos_Id,Proveedores_Id,Paises_Id,Paises2_Id,Ciudades_Id,Ciudades2_Id,ZonasV_Id,Contenidos_Id,Guias_Bultos,Guias_Peso,Guias_PesoVol,Guias_FOB,Clientes_Id,Guias_ExpoTotVta,Guias_ImpoTotVta,Guias_VentaI,kilos_facturables
count,42406.000000,42406.000000,40906.000000,965.000000,41911.000000,40034.000000,789.000000,42406.000000,42406.000000,42405.000000,42364.000000,36225.000000,3.635500e+04,42406.000000,42406.000000,3.000000,42406.0,42380.000000
mean,475317.765175,113.127411,1866.080746,3.821762,64.184415,30.374382,104.114068,125.182451,2.755789,1.499281,19.870580,22.687450,9.645111e+02,40427.835000,192.439414,184.000000,0.0,25.339526
std,41040.008821,40.590487,961.995605,14.560838,39.814481,80.618152,118.518433,143.450013,0.654823,2.854840,88.018156,105.321718,1.467682e+04,2873.223494,333.867240,46.765372,0.0,112.412574
min,404102.000000,24.000000,1226.000000,1.000000,1.000000,1.000000,1.000000,55.000000,1.000000,1.000000,0.050000,0.100000,1.000000e-01,27948.000000,3.200000,130.000000,0.0,0.050000
25%,438798.250000,97.000000,1226.000000,1.000000,24.000000,2.000000,2.000000,65.000000,3.000000,1.000000,1.000000,4.500000,1.000000e+01,40542.000000,56.585000,170.500000,0.0,2.500000
50%,475133.500000,137.000000,1519.000000,1.000000,91.000000,2.000000,58.000000,76.000000,3.000000,1.000000,1.600000,4.500000,1.000000e+01,40542.000000,105.900000,211.000000,0.0,4.500000
75%,508262.750000,137.000000,1547.000000,1.000000,91.000000,2.000000,187.000000,99.000000,3.000000,1.000000,7.500000,12.000000,1.360000e+02,42024.000000,197.860000,211.000000,0.0,12.000000
max,557872.000000,172.000000,7360.000000,91.000000,311.000000,509.000000,510.000000,729.000000,3.000000,134.000000,4338.000000,8304.000000,1.539440e+06,46025.000000,12682.130000,211.000000,0.0,8304.000000


In [80]:
df_guias_impo.describe()

,Guias_Id,Motivos_Id,Proveedores_Id,Paises_Id,Paises2_Id,Ciudades_Id,Ciudades2_Id,ZonasV_Id,Contenidos_Id,Guias_Bultos,Guias_Peso,Guias_PesoVol,Guias_FOB,Clientes_Id,Guias_ExpoTotVta,Guias_ImpoTotVta,Guias_VentaI,kilos_facturables
count,98708.000000,98708.000000,98209.000000,98196.000000,4.00000,4.000000,98699.000000,36403.000000,98708.000000,98253.000000,98264.000000,98125.000000,97431.000000,98708.000000,3.000000,98708.000000,98708.000000,98266.000000
mean,485415.303218,85.590560,2416.476484,77.372021,150.50000,54.500000,1.985218,99.923468,2.999696,2.031307,19.984398,18.049421,1098.285894,36555.284040,442.866667,862.483145,364.304749,23.888487
std,44445.051115,54.394139,959.957661,29.021760,126.13617,89.116029,0.969397,5.496103,0.024653,7.097137,19.925610,28.061448,830.995342,5127.108851,36.030173,2494.369691,316.557835,28.938130
min,406822.000000,36.000000,1226.000000,1.000000,1.00000,2.000000,1.000000,30.000000,1.000000,1.000000,0.000000,0.000000,0.100000,27948.000000,420.000000,0.720000,0.000000,0.000000
25%,446797.750000,36.000000,1830.000000,76.000000,68.50000,2.000000,2.000000,99.000000,3.000000,1.000000,5.000000,4.000000,476.700000,31205.000000,422.100000,456.412500,152.710000,7.000000
50%,486583.500000,36.000000,1830.000000,91.000000,173.00000,14.500000,2.000000,101.000000,3.000000,1.000000,16.000000,11.000000,932.350000,36910.000000,424.200000,744.795000,286.560000,18.500000
75%,525027.250000,136.000000,3780.000000,91.000000,255.00000,67.000000,2.000000,102.000000,3.000000,2.000000,33.000000,24.000000,1510.385000,40882.000000,454.300000,1148.547500,502.730000,37.000000
max,557914.000000,177.000000,7638.000000,254.000000,255.00000,187.000000,206.000000,167.000000,3.000000,510.000000,923.000000,1604.000000,4169.560000,46040.000000,484.400000,476854.950000,25155.590000,1604.000000


**Agrupamos los dos por mes**

In [81]:
df_guias_impo_group_mensual= df_guias_impo.groupby(pd.Grouper(key='Guias_Fecha', axis=0, 
                      freq='M')).sum()
df_guias_expo_group_mensual= df_guias_expo.groupby(pd.Grouper(key='Guias_Fecha', axis=0, 
                      freq='M')).sum()

In [82]:
##Arreglamos el outlier de expo 
num_dic_2022= df_guias_expo_group_mensual.loc['2022-12-31']['Guias_Peso'] + df_guias_expo_group_mensual.loc['2023-12-31']['Guias_Peso'] 
df_guias_expo_group_mensual.loc['2022-12-31','Guias_Peso']= num_dic_2022

In [83]:
df_mensual_comparativo= df_guias_expo_group_mensual[['Guias_Peso']]

In [84]:
df_mensual_comparativo= df_guias_expo_group_mensual[['Guias_Peso']].loc[:'2023-06-30']
df_mensual_comparativo[['Kilos Impo']]= df_guias_impo_group_mensual[['Guias_Peso']].loc[:'2023-06-30']
df_mensual_comparativo.rename(columns={'Guias_Peso': 'Kilos Expo'}, inplace=True)

fig = px.line(df_mensual_comparativo, x=df_mensual_comparativo.index, y=df_mensual_comparativo.columns,
              #width=800, height=400,
              title='Comparativo Mensual Kilos de Carga por Servicio', 
              labels={
                     "Guias_Fecha": "Mes",
                     "value": "Kilos",
                     "variable": ' '
                 })

fig.show()

#### Porcentaje de variación mensual <a id='4'></a>

In [85]:
df_aumento_mensual = df_guias_impo_group_mensual[['Guias_Peso']].loc[:'2023-06-30'].pct_change()
df_aumento_mensual.rename(columns={'Guias_Peso': 'Impo'}, inplace=True)
df_aumento_mensual['Expo']= df_guias_expo_group_mensual[['Guias_Peso']].loc[:'2023-06-30'].pct_change()

df_aumento_mensual= df_aumento_mensual.loc['2019-01-01':]
df_aumento_mensual.rename_axis('Servicio', axis=1, inplace=True)

In [86]:
fig = px.area(df_aumento_mensual, facet_col="Servicio", 
              facet_col_wrap=2, 
              #width=800, height=400,
              title = 'Evolución Porcentaje de Variación Mensual de Kilos por Servicio', 
              labels={
                     "Guias_Fecha": "Mes",
                     "value": "Variación",
                 },)
fig.show()

#### Comparativo Kilos Semanales por Servicio <a id='5'></a>

* El analisis diario se dificulta ya que los fines de semana y feriados no hay guias cargadas, aunque esos dias si hay operaciones y entregas.Por este motivo, hay fechas donde la cantidad de kilos es 0, cuando en realidad las guias si estan en transito. Asimismo, es controversial medir los kilos por dia ya que una carga puede encontrarse en el warehouse de la sede y por asuntos de cupos de importacion o incluso demoras en los vuelos la carga no puede ser subida al vuelo, y aun asi la guia posee fecha anterior al evento. Ya que la fecha de la guia corresponde al momento en el que fue creada y existe un defasaje por los tiempos de tránsito y entrega de cada operacion con la fecha efectiva en que la carga llega a origen. Por ello, considero prudente realizar un análisis semanal, para optimizar la toma de decisiones y gestion de la carga en los vuelos en las distintas areas de operacion tanto de importacion como para exportacion. 

* Para optimizar el análisis, tomaremos datos desde Enero 2019 hasta Enero 2023, de esta forma realizare un analisis en el periodo completo de 5 años que nos permitirá analizar comportamientos y patrones. 

In [87]:
#Definicion del periodo de analisis de 5 años 
df_guias_impo = df_guias_impo.loc[df_guias_impo["Guias_Fecha"].between("2019-01-01", "2023-01-01")]
df_guias_impo.reset_index(inplace=True)

df_guias_expo = df_guias_expo.loc[df_guias_expo["Guias_Fecha"].between("2019-01-01", "2023-01-01")]
df_guias_expo.reset_index(inplace=True)

In [88]:
#agrupamos los datos para ver las distribuciones semanales para impo y expo
df_guias_impo_group_semanal= df_guias_impo.groupby(pd.Grouper(key='Guias_Fecha', axis=0, 
                      freq='W')).sum()
df_guias_expo_group_semanal= df_guias_expo.groupby(pd.Grouper(key='Guias_Fecha', axis=0, 
                      freq='W')).sum()

In [89]:
df_guias_impo_group_mensual.tail()

,Guias_Id,Motivos_Id,Proveedores_Id,Paises_Id,Paises2_Id,Ciudades_Id,Ciudades2_Id,ZonasV_Id,Contenidos_Id,Guias_Bultos,Guias_Peso,Guias_PesoVol,Guias_FOB,Clientes_Id,Guias_ExpoTotVta,Guias_ImpoTotVta,Guias_VentaI,kilos_facturables
Guias_Fecha,,,,,,,,,,,,,,,,,,
2023-03-31,1569239512,234616,6762149.0,223737.0,1.0,27.0,5649.0,72495.0,8712,4381.0,43352.10,40225.02,2259387.66,105767554,420.0,2136770.21,1096771.82,53752.92
2023-04-30,1374993595,190405,5606540.0,204403.0,0.0,0.0,4996.0,59017.0,7587,4291.0,39299.71,32498.00,1973019.39,91070023,0.0,1836567.87,916357.75,46077.71
2023-05-31,1848024034,284446,7959308.0,258196.0,0.0,0.0,6625.0,49969.0,10125,5888.0,51799.50,46124.25,2702582.68,121669039,0.0,2480450.01,1224957.98,61983.35
2023-06-30,1777093978,262431,7802378.0,249619.0,0.0,0.0,6397.0,10328.0,9655,4971.0,49224.46,43423.88,2576068.59,116326445,0.0,2351166.42,1144797.73,59744.25
2023-07-31,1859726611,272780,7828554.0,264790.0,0.0,0.0,6690.0,0.0,10035,5323.0,52103.20,50625.19,2679719.12,122393934,0.0,2534336.85,1278003.77,66848.65


In [90]:
df_semanal_comparativo= df_guias_expo_group_semanal[['Guias_Peso']]
df_semanal_comparativo[['Kilos Impo']]= df_guias_impo_group_semanal[['Guias_Peso']]
df_semanal_comparativo.rename(columns={'Guias_Peso': 'Kilos Expo'}, inplace=True)

fig = px.line(df_semanal_comparativo, x=df_semanal_comparativo.index, y=df_semanal_comparativo.columns,
              title='Comparativo Semanal Kilos de Carga por Servicio', 
              labels={
                     "Guias_Fecha": "Semana",
                     "value": "Kilos",
                     "variable": ' '
                 }, 
              #width=800, height=600
              )

fig.show()

En este gráfico se observa claramente como semana a semana hay grandes variaciones, las mismas son el resultado de diversos factores como: la cantidad de vuelos semanales que se esten coordinando en el momento, los regimenes aduaneros presentes y los limites que presentan para el comercio internacional, el tipo de cambio y la gestión en las distintas sedes: Miami, Shenzhen, Madrid y Amsterdam, donde de acuerdo al volumen, frecuencia de vuelos y cupos enviarán carga hacia Buenos Aires para realizar la nacionalización y liberacion de la carga. Del mismo modo, para exportación dependerá de las condiciones no solo aduaneras sino financieras y bancarias que permitiran o entorpeceran los procesos de exportacion, por ende la disposición del cliente a contratar este tipo de servicios.

#### Porcentaje de Variación semanal <a id='7'></a>

In [91]:
df_aumento_semanal = df_guias_impo_group_semanal[['Guias_Peso']].pct_change()
df_aumento_semanal.rename(columns={'Guias_Peso': 'Impo'}, inplace=True)
df_aumento_semanal['Expo']= df_guias_expo_group_semanal[['Guias_Peso']].pct_change()

df_aumento_semanal.rename_axis('Servicio', axis=1, inplace=True)


fig = px.area(df_aumento_semanal, facet_col="Servicio", 
              facet_col_wrap=2, 
              title = 'Evolución Porcentaje de Variación Semanal de Kilos por Servicio', 
              labels={
                     "Guias_Fecha": "Mes",
                     "value": "Variación",
                 },)
fig.show()

#### Variables exógenas que afectan el análisis <a id='6'></a>

En un pais como Argentina donde la sensibilidad politica y economica es inestable, analizaremos como el cambio en la cotizacion del dolar afecta el servicio, si hay alguna tendencia evidente entre ellas y si existe algun impacto directo en las preferencias del consumidor respecto a los servicios.m

#### Limpieza por Motivos

En esta parte del EDA asignamos cada guia con su motivo de venta correspondiente para conocer un poco mas cada motivo dentro de cada servicio (Importacion o Exportacion)

In [92]:
dic_clientes= dict(df_clientes.values)
dic_ciudades= dict(df_ciudades.values)
dic_proveedores= dict(df_proveedores.values)
dic_motivos= dict(df_motivos[['Motivos_Id', 'MotivosFac_Nombre']].values)
dic_paises_cod= dict(df_paises[['Paises_Id', 'Paises_Codigo']].values )
dic_paises_nombre= dict(df_paises[['Paises_Id', 'Paises_Nombre']].values)

In [93]:
df_guias_expo.columns

Index(['index', 'Guias_Id', 'Guias_Numero', 'Guias_Fecha', 'Motivos_Id',
       'Proveedores_Id', 'Guias_ContraGuia', 'Guias_GuiaMadre',
       'Guias_Tracking', 'Paises_Id', 'Paises2_Id', 'Ciudades_Id',
       'Ciudades2_Id', 'ZonasV_Id', 'ZonasIV_Id', 'Contenidos_Id',
       'Guias_Bultos', 'Guias_Peso', 'Guias_VolLargo', 'Guias_VolAncho',
       'Guias_VolAlto', 'Guias_PesoVol', 'Guias_FOB', 'Clientes_Id',
       'Guias_ExpoTotVta', 'Guias_ImpoTotVta', 'Guias_VentaI',
       'kilos_facturables'],
      dtype='object')

In [94]:
df_guias_expo_new= df_guias_expo[['Guias_Fecha', 'Motivos_Id', 'Guias_Peso',
                                  'Guias_ExpoTotVta', 
                                  'Proveedores_Id','Clientes_Id',
                                  'Paises2_Id','Ciudades_Id']]
df_guias_expo_new[['Servicio']]='Expo'
df_guias_expo_new.reset_index(inplace=True)
df_guias_expo_new.drop(columns='index', inplace=True)
df_guias_expo_new.rename(columns={'Guias_ExpoTotVta': 'Venta', 
                                  'Paises2_Id': 'Pais', 
                                  'Ciudades_Id':'Ciudad',
                                  'Proveedores_Id': 'Proveedor', 
                                  'Clientes_Id': 'Cliente', 
                                  'Motivos_Id':'Motivo'}, inplace=True)

In [95]:
df_guias_expo_new['Motivo']= df_guias_expo_new['Motivo'].map(dic_motivos)
df_guias_expo_new['Cliente']=df_guias_expo_new['Cliente'].map(dic_clientes)
df_guias_expo_new['Proveedor']=df_guias_expo_new['Proveedor'].map(dic_proveedores)
df_guias_expo_new['Ciudad']=df_guias_expo_new['Ciudad'].map(dic_ciudades)
df_guias_expo_new['Paises_Codigo']= df_guias_expo_new['Pais'].map(dic_paises_cod)
df_guias_expo_new['Pais']= df_guias_expo_new['Pais'].map(dic_paises_nombre)
df_guias_expo_new.head()

,Guias_Fecha,Motivo,Guias_Peso,Venta,Proveedor,Cliente,Pais,Ciudad,Servicio,Paises_Codigo
0,2019-08-30,Exporta Simple - Puerta-Aeropuerto,144.0,1298.10,AIR CLASS CARGO S.A.,COLL FLORIT MARIA INES,ESPAÑA,CAPITAL FEDERAL,Expo,ESP
1,2019-01-02,3-Expo - PTY FEDEX,2.8,129.99,FEDERAL EXPRESS CORP.,ALMANZA AGUSTIN,AUSTRALIA,CAPITAL FEDERAL,Expo,ALI
2,2019-01-02,6-Expo-UPS Express,1.0,55.78,UPS DE ARGENTINA S.A.,EDITORIAL PGQ SA,CANADA,CAPITAL FEDERAL,Expo,CAN
3,2019-01-02,2-Expo - PTY FEDEX PAK,1.2,43.60,FEDERAL EXPRESS CORP.,BANCO ITAU ARGENTINA S A,BRASIL,CAPITAL FEDERAL,Expo,BRA
4,2019-01-02,1-Expo - PTY FEDEX Envelope,0.5,22.71,FEDERAL EXPRESS CORP.,NAVICON S.A.,URUGUAY,CAPITAL FEDERAL,Expo,URU


In [96]:
df_guias_impo_new=df_guias_impo[['Guias_Fecha', 'Motivos_Id', 'Guias_Peso',
                                'Proveedores_Id','Clientes_Id', 'Guias_VentaI',
                                'Paises_Id','Ciudades2_Id']]

df_guias_impo_new[['Servicio']]='Impo'

df_guias_impo_new.reset_index(inplace=True)
df_guias_impo_new.dropna(inplace=True)
df_guias_impo_new.drop(columns='index', inplace=True)
df_guias_impo_new.rename(columns={'Guias_VentaI': 'Venta', 
                                  'Paises_Id': 'Pais', 
                                  'Ciudades2_Id':'Ciudad',
                                  'Proveedores_Id': 'Proveedor', 
                                  'Clientes_Id': 'Cliente', 
                                  'Motivos_Id':'Motivo'}, inplace=True)

In [97]:
df_guias_impo_new['Motivo']= df_guias_impo_new['Motivo'].map(dic_motivos)
df_guias_impo_new['Cliente']=df_guias_impo_new['Cliente'].map(dic_clientes)
df_guias_impo_new['Proveedor']=df_guias_impo_new['Proveedor'].map(dic_proveedores)
df_guias_impo_new['Ciudad']=df_guias_impo_new['Ciudad'].map(dic_ciudades)
df_guias_impo_new['Paises_Codigo']= df_guias_impo_new['Pais'].map(dic_paises_cod)
df_guias_impo_new['Pais']= df_guias_impo_new['Pais'].map(dic_paises_nombre)
df_guias_impo_new.head()

,Guias_Fecha,Motivo,Guias_Peso,Proveedor,Cliente,Venta,Pais,Ciudad,Servicio,Paises_Codigo
0,2019-01-02,USA FLAT,13.0,RAICO INTERNATIONAL,HTM S.A.,161.36,ESTADOS UNIDOS,CAPITAL FEDERAL,Impo,EST
1,2019-01-02,NO USAR IB IMPO MUNDIAL Geobox 53,23.0,"EUROPA GEOBOX , S.L.",HTM S.A.,434.01,INGLATERRA,CAPITAL FEDERAL,Impo,ING
2,2019-01-02,USA FLAT,1.0,RAICO INTERNATIONAL,TRACK-MAR SACI,92.83,ESTADOS UNIDOS,CAPITAL FEDERAL,Impo,EST
3,2019-01-02,ORIENTE FEDEX,32.0,TNT ARGENTINA S.R.L.,INDUSTRIAS MOVILIFT S.R.L.,567.92,CHINA,CAPITAL FEDERAL,Impo,CHN
4,2019-01-02,USA FLAT,5.0,RAICO INTERNATIONAL,TRACK-MAR SACI,92.24,ESTADOS UNIDOS,CAPITAL FEDERAL,Impo,EST


### Análisis de los servicios por motivo 

Antes de comenzar a analizar las distribuciones de cada motivo, es prudente observar que representa ese motivo en el servicio y que impacto a nivel de kilos posee. De esta forma, podemos conocer que motivos son mas relevantes que otros y de que forma interactuan entre si. 
Por este motivo, analizare el ultimo año (enero 2022- enero 2023) para conocer que motivos fueron los principales. 

In [98]:
df_servicios_ultimo_ano = pd.concat([df_guias_expo_new, df_guias_impo_new])

In [101]:
fig = px.sunburst(df_servicios_ultimo_ano, path=['Servicio', 'Motivo'], values='Guias_Peso',
                  color='Motivo', hover_data=['Motivo'], 
                  width=900, height=600)
fig.update_layout(title= 'Composición de las ventas por kilo por servicio')
fig.show()

#### Top 10 Motivos 

In [117]:
df_resumen= df_servicios_ultimo_ano.groupby(['Servicio','Motivo'])['Guias_Peso', 'Servicio'].sum().sort_values(by='Guias_Peso', ascending= False)
df_resumen.head(10)

C:\Users\Bravo15\AppData\Local\Temp\ipykernel_17820\1944926756.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Guias_Peso
Servicio Motivo                                              
Impo     USA FLAT                                   671980.80
Expo     Exporta Simple - Puerta-Aeropuerto         448001.95
Impo     ORIENTE UPS                                237058.38
         NO USAR IB IMPO MUNDIAL  Geobox 53         129644.11
         USA INSUA                                  101597.60
Expo     CARGA AEREA EXPO PREPAID                    90786.65
Impo     Courier Oriente FLAT Wish/Latin logistic    87072.31
         TVH PPD USA                                 75605.00
         UPS MERCOSUR                                63206.37
Expo     CARGA AEREA EXPO - Q                        57592.60

Una vez obtenidos los motivos mas relevantes y los mas utilizados por los clientes, se puede observar como algunas rutas son utilizadas con mayor frecuencia que otras. Una ruta esta compuesta de varios motivos, ya que para un mismo origen o destino se pueden elegir entre varios agentes y proveedores de acuerdo a las necesidades del cliente lo que determinara cual sera su motivo de venta. En este sentido, podemos observar como la ruta USA que integra los motivos: 528 USA FLAT, 599 TVH PPD USA, USA MIX y USA INSUA, es la ruta que en su conjunto suma mas.  

#### Top 5 Motivos Impo

In [123]:
df_servicios_ultimo_ano[df_servicios_ultimo_ano['Servicio'] == 'Impo'].groupby(['Motivo'])['Guias_Peso', 'Servicio'].sum().sort_values(by='Guias_Peso', ascending= False).head(5)

C:\Users\Bravo15\AppData\Local\Temp\ipykernel_17820\1798605061.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Guias_Peso
Motivo,
USA FLAT,671980.80
ORIENTE UPS,237058.38
NO USAR IB IMPO MUNDIAL Geobox 53,129644.11
USA INSUA,101597.60
Courier Oriente FLAT Wish/Latin logistic,87072.31


Para analizar cada motivo por separado, creare un dataset por cada uno. 